In [1]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../../../locaTE.jl/")
using Revise

  Activating project at `/data/gpfs/projects/punim0638/stephenz/locaTE-paper/examples/pancreas`
   Resolving package versions...
  No Changes to `/data/gpfs/projects/punim0638/stephenz/locaTE-paper/examples/pancreas/Project.toml`
  No Changes to `/data/gpfs/projects/punim0638/stephenz/locaTE-paper/examples/pancreas/Manifest.toml`


In [2]:
using NPZ
using locaTE
using NearestNeighbors
using Graphs
using GraphSignals
using SparseArrays
using LinearAlgebra
using Plots
using StatsBase
using CUDA
using Random
using NNlib

[ Info: Precompiling locaTE [d47a2e43-82e6-47ba-a340-eddb77145d33]


In [3]:
DATA_DIR = "../../data/pancreas/"
X = npzread(string(DATA_DIR, "X.npy"))
R = sparse(npzread(string(DATA_DIR, "R.npy")))
P = sparse(npzread(string(DATA_DIR, "P.npy")))
QT = to_backward_kernel(P);

In [4]:
disc = locaTE.discretizations_bulk(X);

In [13]:
targets, regulators = 1:size(X, 2), 1:size(X, 2)
i = 1
gene_idxs = vcat([[j, i]' for i in targets for j in regulators]...);
clusters = I(size(P, 1))
clusters_norm = convert(Matrix{eltype(P)}, clusters)
clusters_norm ./= sum(clusters_norm; dims = 1);

In [15]:
using BenchmarkTools

In [16]:
@btime get_MI(X, compute_coupling(X, clusters_norm[:, i], P, QT, R), gene_idxs[:, 1], gene_idxs[:, 2]; disc = disc)

  191.513 s (168340 allocations: 143.53 GiB)


6724-element Vector{Float64}:
 -8.881784197001252e-16
  0.009175991355972801
  0.00224549920634165
  0.0030092356795135
  0.0015914341928116027
  0.008967473929542669
  0.006149163894478615
  0.009095682209627975
  0.014682029927894469
  0.010957401430054414
  0.01234173654665982
  0.007934523673550853
  0.010423357848992731
  ⋮
  0.0024781321685387603
  0.011622785557160897
  0.003407045316749846
  0.012028540615454553
  0.00752909246200173
  0.005041846651464343
  0.005375799071123399
  0.0007775327968284618
  0.009940953490491289
  0.006016697994755571
  0.00537641300859959
 -6.661338147750939e-16

┌ Error: Error watching manifest
│   exception =
│    MethodError: no method matching (::Rmath_jll.var"#make_wrapper_dict#4"{Rmath_jll.var"#parse_wrapper_platform#3"})(::String, ::Vector{String})
│    Stacktrace:
│     [1] top-level scope
│       @ ~/.julia/packages/JLLWrappers/QpMQW/src/toplevel_generators.jl:156
│    Revise evaluation error at /home/stephenz/.julia/packages/JLLWrappers/QpMQW/src/toplevel_generators.jl:156
│    
│    Stacktrace:
│     [1] methods_by_execution!(recurse::Any, methodinfo::Revise.CodeTrackingMethodInfo, docexprs::Dict{Module, Vector{Expr}}, mod::Module, ex::Expr; mode::Symbol, disablebp::Bool, always_rethrow::Bool, kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
│       @ Revise ~/.julia/packages/Revise/P8ITN/src/lowered.jl:227
└ @ Revise ~/.julia/packages/Revise/P8ITN/src/pkgs.jl:477
